In [1]:
sc.stop()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

conf = SparkConf().setAppName("202-f")\
                  .setMaster("spark://10.120.26.200:7077")\
                  .set("spark.executor.memory", '6g')\
                  .set('spark.cores.max', '4')

sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName('sql coming~').getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
58,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
data = spark.read.csv(header=True, path="hdfs://10.120.26.200/user/spark/spark_sql_101/crime/data/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
crime_2015_6 = data.filter("year >= 2015").drop("lsoa_code")
crime_2015_6.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+-----+----+-----+
|             borough|      major_category|      minor_category|value|year|month|
+--------------------+--------------------+--------------------+-----+----+-----+
|               Brent|  Theft and Handling|Motor Vehicle Int...|    0|2015|    9|
|       Tower Hamlets|            Burglary|Burglary in a Dwe...|    0|2016|    3|
|Richmond upon Thames|Violence Against ...|      Common Assault|    0|2015|    3|
|               Brent|  Theft and Handling|Theft/Taking of P...|    0|2015|    1|
|           Redbridge|            Burglary|Burglary in Other...|    0|2015|    2|
|             Bromley|     Criminal Damage|Criminal Damage T...|    0|2015|    9|
|          Hillingdon|Violence Against ...|        Wounding/GBH|    1|2015|    6|
|              Ealing|  Theft and Handling|  Other Theft Person|    0|2015|    8|
|           Greenwich|     Criminal Damage|Criminal Damage T...|    0|2015|    1|
|           Redb

In [4]:
convictions_by_borough = crime_2015_6.groupBy("borough").agg({"value": "sum"})
convictions_by_borough.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+
|             borough|sum(value)|
+--------------------+----------+
|             Croydon|    5669.0|
|          Wandsworth|    4791.0|
|              Bexley|    2513.0|
|             Lambeth|    6837.0|
|Barking and Dagenham|    3310.0|
|              Camden|    5790.0|
|           Greenwich|    4413.0|
|              Newham|    5807.0|
|       Tower Hamlets|    5855.0|
|              Barnet|    4960.0|
|            Hounslow|    4311.0|
|              Harrow|    2552.0|
|Kensington and Ch...|    3973.0|
|           Islington|    5541.0|
|               Brent|    5327.0|
|            Haringey|    5194.0|
|             Bromley|    3978.0|
|              Merton|    2336.0|
|         Westminster|    9391.0|
|           Southwark|    6284.0|
+--------------------+----------+
only showing top 20 rows

In [5]:
convictions_by_borough = convictions_by_borough.withColumnRenamed("sum(value)", "num_of_convictions")
convictions_by_borough.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|             borough|num_of_convictions|
+--------------------+------------------+
|             Croydon|            5669.0|
|          Wandsworth|            4791.0|
|              Bexley|            2513.0|
|             Lambeth|            6837.0|
|Barking and Dagenham|            3310.0|
|              Camden|            5790.0|
|           Greenwich|            4413.0|
|              Newham|            5807.0|
|       Tower Hamlets|            5855.0|
|              Barnet|            4960.0|
|            Hounslow|            4311.0|
|              Harrow|            2552.0|
|Kensington and Ch...|            3973.0|
|           Islington|            5541.0|
|               Brent|            5327.0|
|            Haringey|            5194.0|
|             Bromley|            3978.0|
|              Merton|            2336.0|
|         Westminster|            9391.0|
|           Southwark|            6284.0|
+--------------------+------------

In [6]:
convictions_by_borough = convictions_by_borough.withColumnRenamed("sum(value)", "num_of_convictions")
convictions_by_borough.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|             borough|num_of_convictions|
+--------------------+------------------+
|             Croydon|            5669.0|
|          Wandsworth|            4791.0|
|              Bexley|            2513.0|
|             Lambeth|            6837.0|
|Barking and Dagenham|            3310.0|
|              Camden|            5790.0|
|           Greenwich|            4413.0|
|              Newham|            5807.0|
|       Tower Hamlets|            5855.0|
|              Barnet|            4960.0|
|            Hounslow|            4311.0|
|              Harrow|            2552.0|
|Kensington and Ch...|            3973.0|
|           Islington|            5541.0|
|               Brent|            5327.0|
|            Haringey|            5194.0|
|             Bromley|            3978.0|
|              Merton|            2336.0|
|         Westminster|            9391.0|
|           Southwark|            6284.0|
+--------------------+------------

In [7]:
total_convictions = convictions_by_borough.agg({"num_of_convictions": "sum"}).collect()[0][0]
total_convictions

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

145999.0

In [8]:
convictions_by_borough_with_percentage = convictions_by_borough.withColumn("percentage_convictions",
                            format_number(convictions_by_borough["num_of_convictions"] / total_convictions * 100, 2))
convictions_by_borough_with_percentage

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[borough: string, num_of_convictions: double, percentage_convictions: string]

In [9]:
convictions_by_borough_with_percentage.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[borough: string, num_of_convictions: double, percentage_convictions: string]

In [10]:
# show result of 100 records to console
convictions_by_borough_with_percentage.show(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+----------------------+
|             borough|num_of_convictions|percentage_convictions|
+--------------------+------------------+----------------------+
|             Croydon|            5669.0|                  3.88|
|          Wandsworth|            4791.0|                  3.28|
|              Bexley|            2513.0|                  1.72|
|             Lambeth|            6837.0|                  4.68|
|Barking and Dagenham|            3310.0|                  2.27|
|              Camden|            5790.0|                  3.97|
|           Greenwich|            4413.0|                  3.02|
|              Newham|            5807.0|                  3.98|
|       Tower Hamlets|            5855.0|                  4.01|
|              Barnet|            4960.0|                  3.40|
|            Hounslow|            4311.0|                  2.95|
|              Harrow|            2552.0|                  1.75|
|Kensington and Ch...|   

In [2]:
# 連資料庫
# convictions_by_borough_with_percentage.write \
#                                           .option("driver", "com.mysql.jdbc.Driver") \
#                                           .jdbc("jdbc:mysql://localhost:3306", "crime.convictions_by_borough_with_percentage",
#                                                   properties={"user": "spark", "password": "spark"},mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…